# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to Ollama Python Client.
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Chat API : Ollama

Let's start with a Ollama local LLMs

In [1]:
import ollama

ollama.chat(model='qwen2:1.5b', messages=[{'role': 'user', 'content': 'Why is the sky blue?'}])

{'model': 'qwen2:1.5b',
 'created_at': '2024-10-16T17:51:39.5558359Z',
 'message': {'role': 'assistant',
  'content': "The color of the sky is actually not blue, but a shade of blue that people observe when looking up at the clear, sunny sky. This is because sunlight contains different colors of light and our eyes are sensitive to certain wavelengths more than others.\nWhen we look at the sky, the sunlight passing through Earth's atmosphere interacts with molecules in the air (like water droplets or dust particles) and other gases. The wavelength of light that is refracted (bent) by these particles can be scattered into different colors depending on their speed compared to the speed of sound within the air.\nAt sea level, at about 3600 feet above ground, sunlight reaches an altitude where the blue wavelengths have been scattered more than other longer wavelength colors. This causes the sky and clouds to appear blue to us. However, as we move higher up into the atmosphere, the concentra

In [2]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model = "qwen2:1.5b",temperature = 0) # format = 'json' is also an argument 

llm.invoke("Tell me a joke")

AIMessage(content="Sure, here's one for you:\nWhy was the math book sad?\nBecause it had too many problems.", response_metadata={'model': 'qwen2:1.5b', 'created_at': '2024-10-16T17:52:59.9746319Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1270613600, 'load_duration': 82603300, 'prompt_eval_count': 12, 'prompt_eval_duration': 282086000, 'eval_count': 23, 'eval_duration': 901870000}, id='run-fa3af98c-c28b-41a0-b85d-d7b6a4e01ebf-0')

In [3]:
llm.invoke("What is 1+1?")

AIMessage(content='The sum of one plus one is two.', response_metadata={'model': 'qwen2:1.5b', 'created_at': '2024-10-16T17:53:11.2041666Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 803334300, 'load_duration': 88441900, 'prompt_eval_count': 15, 'prompt_eval_duration': 390975000, 'eval_count': 10, 'eval_duration': 319764000}, id='run-83e76b7b-5f2f-458d-87fc-aaf40a860307-0')

### Prompt template

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

chat = ChatOllama(model = "qwen2:1.5b",temperature = 0) # format = 'json' is also an argument 

In [33]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [34]:
print("The prompt template:",prompt_template.messages[0].prompt)

print("Input variables of prompt: ", prompt_template.messages[0].prompt.input_variables)

The prompt template: input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
Input variables of prompt:  ['style', 'text']


In [35]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [36]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [37]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [38]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [39]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

print(customer_response.content)

"Ah, I'm seething that my blender lid flew off and splattered my kitchen walls with smoothie!" said the speaker in a calm and collected manner. "And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your assistance right now, matey!" The text was written using triple backticks, which is commonly used in programming languages to denote code snippets or command line input/output. However, this particular text appears to be a humorous and lighthearted statement, likely meant to evoke the spirit of pirates or sailors from the past. In any case, the speaker's request for help is clear and urgent, and their tone is polite and respectful.


In [40]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [41]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [42]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Ahoy there me hearties, 'tis ye, the customer, and we be lettin' ye know that the warranty don't cover the cost of cleanin' yer kitchen after ye misused yer blender by forgettin' to put the lid on before startin' it. Tough luck, matey! We'll see ye later!

(Note: The text is written in a humorous and exaggerated English Pirate style, but still maintains a polite tone.)


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [43]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [44]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [45]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [46]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOllama(temperature=0.0, model="zephyr")
response = chat(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there",
    "worth it for the extra features"
  ]
}


In [47]:
type(response.content)

str

In [56]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [48]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [49]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [50]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [53]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [54]:
response = chat.invoke(messages)

print(response.content)

```json
{
  "gift": false,
  "delivery_days": 2,
  "price_value": [
    "This leaf blower is slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


In [55]:
output_dict = output_parser.parse(response.content)

print(output_dict)

{'gift': False, 'delivery_days': 2, 'price_value': ["This leaf blower is slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


In [57]:
type(output_dict)

output_dict.get('delivery_days')

2